In [1]:
import tweepy
import configparser
import pandas as pd

In [2]:
# read configs
config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

In [3]:
# authentication
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

### Get tweet with specific keywords

In [4]:
# search tweets
keywords = 'vaksin booster'
limit = 5000

tweet = tweepy.Cursor(api.search_tweets, q=keywords, count=100, tweet_mode='extended').items(limit)

# for tweet in tweet:
#     if len(tweet.entities["user_mentions"]) != 0:
#         screen_name = tweet.entities["user_mentions"][0]["screen_name"]
#         user = api.get_user(screen_name=screen_name)
#         print(user.followers_count)

In [5]:
# create dataframe

columns = ["created_at", 'User', 'Tweet', "User's_Followers", "mentioned_user", "mentioned_user_followers"]
data = []

for tweet in tweet:
    if len(tweet.entities["user_mentions"]) != 0:
#         get mentioned user's followers count
        screen_name = tweet.entities["user_mentions"][0]["screen_name"]
        try:
            user = api.get_user(screen_name=screen_name)
            men_us_foll = user.followers_count
        except:
            men_us_foll = 0
            
        data.append([tweet.created_at, tweet.user.screen_name, tweet.full_text, tweet.user.followers_count, screen_name, men_us_foll])
    
df = pd.DataFrame(data, columns=columns)
df

,created_at,User,Tweet,User's_Followers,mentioned_user,mentioned_user_followers
0,2022-05-22 04:54:56+00:00,cingheuzzz,"Info vaksin booster yg bisa hari minggu, diman...",107,infobdg,2564315
1,2022-05-22 04:48:41+00:00,GOONJOL,RT @we_aremania: Aremania yang ingin nonton la...,503,we_aremania,193815
2,2022-05-22 04:37:07+00:00,gemarbernafas,@PRFMnews mohon infonya vaksin booster sinovac...,52,PRFMnews,284661
3,2022-05-22 04:32:49+00:00,rumahsakit_ui,@faradinamzr_ @yaelahqie Selamat siang Kak. Se...,2621,faradinamzr_,733
4,2022-05-22 04:20:38+00:00,KantGus,RT @forjakeu: Pemerintah telah memilih Vaksin ...,2566,forjakeu,8592
...,...,...,...,...,...,...
3512,2022-05-15 01:18:32+00:00,arearaa,RT @yuno_yejun: ADOPSI KUCING KITTEN (FREE)\n✨...,1101,yuno_yejun,1097
3513,2022-05-15 01:18:06+00:00,KangCendol_33,RT @Penyair_Berdiri: Vaksin Booster Aman &amp;...,11304,Penyair_Berdiri,19125
3514,2022-05-15 01:18:02+00:00,KangCendol_33,RT @Penyair_Berdiri: Seluruh vaksin covid yang...,11304,Penyair_Berdiri,19125
3515,2022-05-15 01:17:11+00:00,KangCendol_33,"RT @catatankaqihati: Vaksin Booster Aman, dn n...",11304,catatankaqihati,10801


In [6]:
import numpy as np

earliest_date = df["created_at"].sort_values().head(1).values[0]
newest_date = df["created_at"].sort_values().tail(1).values[0]

print(f'Earliest date: \n{earliest_date}')
print()
print(f'Newest date: \n{newest_date}')

# The problem is: the tweet collected is only from one day. How to get max tweet per day = 100 ?
# so that the result would not bias.

Earliest date: 
2022-05-15T01:17:07.000000000

Newest date: 
2022-05-22T04:54:56.000000000


In [7]:
df.to_csv("tw1_22-05-2022.csv", sep=",", index=False)

### SNA Feature Engineering

In [7]:
df_interaction = df.drop(['Tweet', 'created_at'], axis=1)
df_interaction

,User,User's_Followers,mentioned_user,mentioned_user_followers
0,KAI121,1024905,unfunded92,306
1,waram,389,sichtng,737
2,raranissa,89,contactap2,37089
3,CILA_capt,39,TheNugrawesome,12333
4,arindrabara,64,sbmptnfess,421735
...,...,...,...,...
558,apolloresor,453,forjakeu,8592
559,rajainfo1,31,Zyidan16,1
560,AnresGunawan_,82,heyydriiii,64
561,kalla_jengking,1872,forjakeu,8592


In [8]:
df_interaction.to_csv("interaction_20-05-2022.csv", sep=",",index=False)

In [9]:
df_edges = df.drop(['Tweet', 'created_at', "User's_Followers", "mentioned_user_followers"], axis=1)
df_edges

,User,mentioned_user
0,KAI121,unfunded92
1,waram,sichtng
2,raranissa,contactap2
3,CILA_capt,TheNugrawesome
4,arindrabara,sbmptnfess
...,...,...
558,apolloresor,forjakeu
559,rajainfo1,Zyidan16
560,AnresGunawan_,heyydriiii
561,kalla_jengking,forjakeu


In [10]:
df_edges.to_csv("edges_19-05-2022.csv", index=True)